In [50]:
import pandas as pd

standard_amino_acid_codes = [
    'A', 'R', 'N', 'D', 'B', 'C', 'E', 'Q', 'Z', 'G', 'H', 'I', 'L', 'K', 
    'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V'
]

def filtered(item):
    modded_item = str(item)
    for char in str(item):
        if char not in standard_amino_acid_codes:
            modded_item = modded_item.replace(char, "X")
            if char == 'U':
                print(item)
                print(modded_item)
        if char.islower():
            return ""
    return str(modded_item).replace("\n", "").replace("*", "").strip().replace(" ", "") + "|"


# Read the data from the xlsx file
df = pd.read_excel('Luzeena_sORF_Database_2024.xlsx')

list_ = df["ORF sequence"].values

filtered_ = [filtered(item) for item in list_ if filtered(item) != ""]

print(len(filtered_))   
filtered_ = list(set(filtered_))
print(len(filtered_))   

# Save the filtered to a txt file 
with open("filtered.txt", "w") as f:
    for item in filtered_:
        f.write("%s\n" % item)

9466
9369


In [51]:
with open("filtered.txt", "r") as f:
    seqs = f.readlines()

unique_chars = set()
num = 0
max_len = 0

for seq in seqs:
    if len(seq) > max_len:
        max_len = len(seq)
    for char in seq:
        # if char is lowercase
        if char.islower():
            num += 1
        if char != "\n":
            unique_chars.add(char)

print(num/len(seqs))
print(unique_chars)
print(max_len)

# Make the chars into one hot encodings
char_to_int = dict((c, i) for i, c in enumerate(unique_chars))
int_to_char = dict((i, c) for i, c in enumerate(unique_chars))

# Use torch to convert a sequence to a matrix of one hot encodings
import torch

def seq_to_one_hot(seq, char_to_int):
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in seq]
    # one hot encode
    onehot_encoded = list()
    for value in integer_encoded:
        letter = [0 for _ in range(len(char_to_int))]
        letter[value] = 1
        onehot_encoded.append(letter)
    return torch.tensor(onehot_encoded)

# Test the function
seq = "CHKSE"
one_hot = seq_to_one_hot(seq, char_to_int)

print(one_hot)

def argmax_one_hot_to_seq(one_hot, int_to_char):
    seq = ""
    for one_hot_vec in one_hot:
        seq += int_to_char[torch.argmax(one_hot_vec).item()]
    return seq

# Test the function
seq = argmax_one_hot_to_seq(one_hot / 2 + 0.1, int_to_char)
print(seq)

0.0
{'C', 'H', 'K', '|', 'S', 'E', 'D', 'Q', 'I', 'Y', 'G', 'N', 'F', 'P', 'X', 'T', 'M', 'W', 'R', 'V', 'L', 'A'}
60
tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
CHKSE


In [52]:
# For the "normal length" human proteome

# Read the data from the tsv file
df = pd.read_csv('uniprotkb_proteome_UP000005640_2024_08_14.tsv', sep="\t")

def filtered2(item):
    for char in item:
        if char not in unique_chars:
            return ""
    return item


human_prot = list(set(df["Sequence"].values))

print(len(human_prot))
human_prot = [filtered(item) for item in human_prot if filtered(item) != ""]
print(len(human_prot))

# Save the filtered to a txt file
with open("human_prot.txt", "w") as f:
    for item in human_prot:
        f.write("%s\n" % item)

82557
...
82557


In [54]:
with open("human_prot.txt", "r") as f:
    seqs = f.readlines()

unique_chars_hum = set()
num = 0
max_len = 0

for seq in seqs:
    if len(seq) > max_len:
        max_len = len(seq)
    for char in seq:
        # if char is lowercase
        if char.islower():
            num += 1
        if char != "\n":
            unique_chars_hum.add(char)

print(num/len(seqs))
print(unique_chars_hum)
print(unique_chars)
print(max_len)

0.0
{'C', 'H', 'K', '|', 'S', 'E', 'D', 'Q', 'I', 'Y', 'G', 'N', 'F', 'P', 'X', 'T', 'M', 'W', 'R', 'V', 'L', 'A'}
{'C', 'H', 'K', '|', 'S', 'E', 'D', 'Q', 'I', 'Y', 'G', 'N', 'F', 'P', 'X', 'T', 'M', 'W', 'R', 'V', 'L', 'A'}
35993


In [ ]:
random_seed = 42

